This first code is for the Sanity check Function

In [ ]:
#6
import pandas as pd

# This Function is for battery_voltage and solar_voltage
def check_voltage(data, column, max_val, min_val, suspicious_threshold=40):
    # Initialize all flags
    data[f'{column}_garbage'] = False
    data[f'{column}_tocheck'] = False

    # First pass: Mark initial garbage based on checks
    for i in range(1, len(data)):  # Start from 1 since we compare with the previous row
        current_val = data.at[i, column]
        previous_val = data.at[i - 1, column]

        # Assume not garbage initially
        garbage_flag = False
        
        # Check range
        if current_val > max_val or current_val < min_val or current_val <= suspicious_threshold:
            garbage_flag = True
        else:
            # Check percentage difference with the previous value, ignoring if the previous value is zero
            if previous_val != 0:
                percentage_diff = abs((current_val - previous_val) / previous_val) * 100
                if percentage_diff > 50:
                    garbage_flag = True

        # Apply garbage flag in the first pass
        data.at[i, f'{column}_garbage'] = garbage_flag
        
         
# Second pass: Mark middle values as garbage if they are between two garbage values
    for i in range(1, len(data) - 1):
        if data.at[i - 1, f'{column}_garbage'] and data.at[i + 1, f'{column}_garbage']:
            data.at[i, f'{column}_garbage'] = True
    # Create the sanity-checked column
    data[f'sanity_checked_{column}'] = data.apply(
        lambda row: 0 if row[f'{column}_garbage'] else row[column], axis=1
    )

# This one is more universal / allgemein: used in ['temp_out', 'humid_in', 'temp_in']
def check_garbage(data, column, max_val, min_val):
    data[f'{column}_garbage'] = False
    data[f'{column}_tocheck'] = False

    for i in range(len(data)):
        current_val = data.at[i, column]

        # Check for garbage based on value range
        if current_val > max_val or current_val < min_val:
            data.at[i, f'{column}_garbage'] = True

    # Create the sanity-checked column
    data[f'sanity_checked_{column}'] = data.apply(
        lambda row: 0 if row[f'{column}_garbage'] else row[column], axis=1)

# Function with diff_value logic for battery_current
def check_battery_current(data, column, max_val, min_val, diff_value_threshold=50):
    data[f'{column}_garbage'] = False

    for i in range(1, len(data)):  # Start from 1 to ensure there's a previous value
        current_val = data.at[i, column]
        previous_val = data.at[i - 1, column]

        # Assume not garbage initially
        garbage_flag = current_val > max_val or current_val < min_val

        # Check if the difference with previous value exceeds 50%
        if not garbage_flag:
            abs_diff_prev = abs(current_val - previous_val)
            percentage_diff_prev = abs((current_val - previous_val) / max(previous_val, 1)) * 100
            if percentage_diff_prev > 50 and abs_diff_prev > diff_value_threshold:
                garbage_flag = True

        # Apply garbage flag
        data.at[i, f'{column}_garbage'] = garbage_flag
        
    # Second pass: Mark middle values as garbage if they are between two garbage values
    for i in range(1, len(data) - 1):
        if data.at[i - 1, f'{column}_garbage'] and data.at[i + 1, f'{column}_garbage']:
            data.at[i, f'{column}_garbage'] = True
    

    # Create the sanity-checked column
    data[f'sanity_checked_{column}'] = data.apply(
        lambda row: 0 if row[f'{column}_garbage'] else row[column], axis=1
    )
    
# solar_power with threshold for percentage difference
def check_solar_power(data, column, max_val, min_val, diff_value_threshold=10):
    data[f'{column}_garbage'] = False
    data[f'{column}_tocheck'] = False

    for i in range(1, len(data)):  # Start from 1 since we only compare with the previous row
        current_val = data.at[i, column]
        previous_val = data.at[i - 1, column]

        # Check for garbage based on value range
        if current_val > max_val or current_val < min_val:
            data.at[i, f'{column}_garbage'] = True
        else:
            # Check for 50% difference with the previous row only if abs diff exceeds threshold
            abs_diff_prev = abs(current_val - previous_val)
            if abs_diff_prev >= diff_value_threshold:
                percentage_diff_prev = abs((current_val - previous_val) / max(previous_val, 1)) * 100
                if percentage_diff_prev > 50:
                    data.at[i, f'{column}_garbage'] = True

    # Second pass: Garbage marking for values between garbage rows
    for i in range(1, len(data) - 1):
        if data.at[i - 1, f'{column}_garbage'] and data.at[i + 1, f'{column}_garbage']:
            data.at[i, f'{column}_garbage'] = True
                        
    # Create the sanity-checked column
    data[f'sanity_checked_{column}'] = data.apply(
        lambda row: 0 if row[f'{column}_garbage'] else row[column], axis=1
    )
    
    
# Function for solar_energy_day and solar_energy_total
def check_solar_energy(data, column, max_val, min_val):
    data[f'{column}_garbage'] = False

    # First pass: Initial marking based on value range and percentage difference from the previous row
    for i in range(1, len(data)):  # Start from 1 since we compare with the previous row
        current_val = data.at[i, column]
        previous_val = data.at[i - 1, column]
        garbage_flag = False

        # Check for garbage based on the value range
        if current_val > max_val or current_val < min_val:
            garbage_flag = True
        else:
            # Check for 50% difference with the previous row
            percentage_diff_prev = abs((current_val - previous_val) / max(previous_val, 1)) * 100
            if percentage_diff_prev > 50:
                garbage_flag = True

        # Apply garbage flag
        data.at[i, f'{column}_garbage'] = garbage_flag

    # Second pass: Ensure values between garbage rows are marked as garbage
    for i in range(1, len(data) - 1):
        if data.at[i - 1, f'{column}_garbage'] and data.at[i + 1, f'{column}_garbage']:
            data.at[i, f'{column}_garbage'] = True
    
    # Create the sanity-checked column
    data[f'sanity_checked_{column}'] = data.apply(lambda row: 0 if row[f'{column}_garbage'] else row[column], axis=1)

    

# Load the CSV file
file_path = 'C:/Users/tloe2/Downloads/core_values_box-0_dist-113 (1).csv'
data = pd.read_csv(file_path)

# Define parameters for each column to check
parameters = {
    'solar_voltage': {'MAX': 25350, 'MIN': 0},
    'humid_in': {'MAX': 100, 'MIN': 0},
    'temp_in': {'MAX': 100, 'MIN': -10},
    'temp_out': {'MAX': 50, 'MIN': -10},  # Adjusted realistic range for temperature
    'solar_power': {'MAX': 150, 'MIN': 0},
    'solar_energy_day': {'MAX': 312, 'MIN': 0},
    'solar_energy_total': {'MAX': 2258, 'MIN': 0},
    'battery_voltage': {'MAX': 14000, 'MIN': 10000},
    'battery_current': {'MAX': 10000, 'MIN': -1000},
}

# Apply the logic to each column
for column, params in parameters.items():
    if column == ['solar_voltage', 'battery_voltage']:
        check_voltage(data, column, params['MAX'], params['MIN'], suspicious_threshold=40)
    elif column == 'solar_power':
        check_solar_power(data, column, params['MAX'], params['MIN'], diff_value_threshold=10)   
    elif column in ['solar_energy_day', 'solar_energy_total']:
        check_solar_energy(data, column, params['MAX'], params['MIN'])     
    elif column == 'battery_current':
        check_battery_current(data, column, params['MAX'], params['MIN'], diff_value_threshold=50)
    else:
        check_garbage(data, column, params['MAX'], params['MIN'])

# Save the updated CSV file
output_file = 'C:/Users/tloe2/Downloads/sanity_checked_core_values_box_0.csv'
data.to_csv(output_file, index=False)

print(f"Sanity checked CSV file saved to: {output_file}")

And then this other code is to format the csv to line protocol format thats more accepted in InfluxDB

In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'C:/Users/tloe2/Downloads/sanity_checked_core_values_box_0.csv'
data = pd.read_csv(file_path)

# Define the static tags
measurement = "core_values"
box_id = 0
dev_type = 0
dist_id = 113

# Scaling factors
scaling_factors = {
    'solar_voltage': 1000,
    'solar_energy_day': 100,
    'solar_energy_total': 100,
    'battery_voltage': 1000,
    'battery_current': 1000
}

# Create a list to hold the Annotated CSV rows
annotated_rows = []

# Iterate through the data and process each row
for index, row in data.iterrows():
    # Skip rows with missing unix_time
    if pd.isna(row['unix_time']):
        continue

    # Exclude rows with any garbage columns set to TRUE
    garbage_columns = [
        'humid_in_garbage', 'temp_in_garbage', 'temp_out_garbage',
        'solar_voltage_garbage', 'solar_power_garbage', 
        'solar_energy_day_garbage', 'solar_energy_total_garbage',
        'battery_voltage_garbage','battery_current_garbage'
    ]
    if any(row[col] == True for col in garbage_columns):
        continue

    # Use sanity_checked columns and scale them if needed
    humid_in = row['sanity_checked_humid_in'] if pd.notna(row['sanity_checked_humid_in']) else 0
    temp_in = row['sanity_checked_temp_in'] if pd.notna(row['sanity_checked_temp_in']) else 0
    temp_out = row['sanity_checked_temp_out'] if pd.notna(row['sanity_checked_temp_out']) else 0
    solar_voltage = (row['sanity_checked_solar_voltage'] / scaling_factors['solar_voltage']) if pd.notna(row['sanity_checked_solar_voltage']) else 0
    solar_power = row['sanity_checked_solar_power'] if pd.notna(row['sanity_checked_solar_power']) else 0
    solar_energy_day = (row['sanity_checked_solar_energy_day'] / scaling_factors['solar_energy_day']) if pd.notna(row['sanity_checked_solar_energy_day']) else 0
    solar_energy_total = (row['sanity_checked_solar_energy_total'] / scaling_factors['solar_energy_total']) if pd.notna(row['sanity_checked_solar_energy_total']) else 0
    battery_voltage = (row['sanity_checked_battery_voltage'] / scaling_factors['battery_voltage']) if pd.notna(row['sanity_checked_battery_voltage']) else 0
    battery_current = (row['sanity_checked_battery_current'] / scaling_factors['battery_current']) if pd.notna(row['sanity_checked_battery_current']) else 0

    # Build the tags section
    tags = f"box_id={box_id},dev_type={dev_type},dist_id={dist_id}"
    
    # Build the fields section
    fields = f"humid_in={humid_in:.2f},temp_in={temp_in:.2f},temp_out={temp_out:.2f}," \
             f"solar_voltage={solar_voltage:.3f},solar_power={solar_power}," \
             f"solar_energy_day={solar_energy_day:.2f},solar_energy_total={solar_energy_total:.2f}," \
             f"battery_voltage={battery_voltage:.3f},battery_current={battery_current:.3f}"
    
    # Add the timestamp (in Unix format)
    timestamp = int(row['unix_time'] * 1_000_000_000)
    
    # Combine all parts into a single row
    annotated_row = f"{measurement},{tags} {fields} {timestamp}"
    annotated_rows.append(annotated_row)

# Save the Annotated CSV to a file with Unix line endings
output_file = 'C:/Users/tloe2/Downloads/annotated_core_values_box_0_V3.csv'
with open(output_file, 'w', newline='\n') as f:
    f.write("\n".join(annotated_rows))

print(f"Annotated CSV file saved to: {output_file}")
